# Notebook Valentin

In [1]:
import pandas as pd

##  Intro

### SAA t-1
Let's say we are at time t, we have our fifty states for which we want to prescribe tests with SAA using information of tests sold in time t-1.
Two approches:
- scalar: for a state, take the average on all states for previous time
- vector: points are now vector in dimensions R^50 (for 50 states for instance). Take the vector average, which basicaly mean as we only have a single vector - the one for t-1 - to take exactly the same info as t-1 for each state 

Observe: for scalar: all states will prescribe the same value, namely the average for t-1, whereas it's not the case for vector SAA.


### SAA cumulative

Again there are the two approches: 
- scalar: for each state take the average for all states on all previous step. Again all prescription will thus be the same
- vector: vector average, i.e. here for a given state take the average on all previous step of test stold FOR THIS GIVEN state

## SAA t-1: SAA prescription with training data from t-1 only

In [197]:
data = pd.read_csv('data/final_df_drop.csv')
data.head()

,State,Date,population,confirmed,deaths,incident_rate,mortality_rate,testing_rate,TestsReported
0,Alabama,2020-04-12,4903185.0,3563,93,75.99,2.61,460.30,2165
1,Alabama,2020-04-13,4903185.0,3734,99,79.63,2.65,622.36,2626
2,Alabama,2020-04-14,4903185.0,3953,114,84.31,2.88,706.29,2942
3,Alabama,2020-04-15,4903185.0,4075,118,86.91,2.90,726.76,3289
4,Alabama,2020-04-16,4903185.0,4345,133,92.67,3.06,776.11,3079


In [198]:
# Sort by date
data.sort_values(by='Date', inplace=True)

In [199]:
data.shape

(7176, 9)

In [200]:
### df for SAA t-1 prescription

# Scalar SAA t-1

SAA_1 = data[['State', 'Date', 'TestsReported']]
# Shift Date to compute the average on t-1
SAA_1['Date'] = pd.to_datetime(SAA_1['Date'])
SAA_1['Shift_Date'] = SAA_1['Date'] + pd.Timedelta(days=1)
average = SAA_1.groupby('Shift_Date')['TestsReported'].mean().round().rename('SAA_t_1_scalar')
# Merge df and averages
SAA_1 = SAA_1.merge(average, left_on='Date', right_on='Shift_Date', how='left')
SAA_1

/var/folders/m7/tssqlvyd6qg7dyx5bvdn0x4h0000gn/T/ipykernel_53394/3336101104.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SAA_1['Date'] = pd.to_datetime(SAA_1['Date'])
/var/folders/m7/tssqlvyd6qg7dyx5bvdn0x4h0000gn/T/ipykernel_53394/3336101104.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  SAA_1['Shift_Date'] = SAA_1['Date'] + pd.Timedelta(days=1)


,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar
0,Alabama,2020-04-12,2165,2020-04-13,NaN
1,Louisiana,2020-04-12,1914,2020-04-13,NaN
2,Arizona,2020-04-12,1119,2020-04-13,NaN
3,Rhode Island,2020-04-12,1059,2020-04-13,NaN
4,Maryland,2020-04-12,3015,2020-04-13,NaN
...,...,...,...,...,...
7171,New Hampshire,2020-08-27,3952,2020-08-28,19515.0
7172,Nevada,2020-08-27,7673,2020-08-28,19515.0
7173,Nebraska,2020-08-27,5925,2020-08-28,19515.0
7174,Oregon,2020-08-27,6098,2020-08-28,19515.0


In [201]:
# Vector SAA t-1
SAA_1.sort_values(by=['State', 'Date'], inplace=True)
SAA_1['SAA_t_1_vector'] = SAA_1.groupby('State')['TestsReported'].shift(1)
SAA_1.sort_index(axis=0, inplace=True)
SAA_1

,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar,SAA_t_1_vector
0,Alabama,2020-04-12,2165,2020-04-13,NaN,NaN
1,Louisiana,2020-04-12,1914,2020-04-13,NaN,NaN
2,Arizona,2020-04-12,1119,2020-04-13,NaN,NaN
3,Rhode Island,2020-04-12,1059,2020-04-13,NaN,NaN
4,Maryland,2020-04-12,3015,2020-04-13,NaN,NaN
...,...,...,...,...,...,...
7171,New Hampshire,2020-08-27,3952,2020-08-28,19515.0,4619.0
7172,Nevada,2020-08-27,7673,2020-08-28,19515.0,5516.0
7173,Nebraska,2020-08-27,5925,2020-08-28,19515.0,5288.0
7174,Oregon,2020-08-27,6098,2020-08-28,19515.0,7248.0


In [202]:
# Remove t_0 as only NaN SAA prescritions
#SAA_1.dropna(inplace=True)

In [203]:
SAA_1

,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar,SAA_t_1_vector
0,Alabama,2020-04-12,2165,2020-04-13,NaN,NaN
1,Louisiana,2020-04-12,1914,2020-04-13,NaN,NaN
2,Arizona,2020-04-12,1119,2020-04-13,NaN,NaN
3,Rhode Island,2020-04-12,1059,2020-04-13,NaN,NaN
4,Maryland,2020-04-12,3015,2020-04-13,NaN,NaN
...,...,...,...,...,...,...
7171,New Hampshire,2020-08-27,3952,2020-08-28,19515.0,4619.0
7172,Nevada,2020-08-27,7673,2020-08-28,19515.0,5516.0
7173,Nebraska,2020-08-27,5925,2020-08-28,19515.0,5288.0
7174,Oregon,2020-08-27,6098,2020-08-28,19515.0,7248.0


In [204]:
#SAA_1.to_csv('data/prescription/saa_t_1.csv', index=None)

## SAA cumul: SAA prescription with cumulative training data

In [215]:
SAA_1 = saa.copy()

In [216]:
# Scalar SAA cumul

SAA_1['Day_before'] = SAA_1['Date'] + pd.Timedelta(days=-1)
n = len(SAA_1[SAA_1.Date == '2020-04-12'])
cumulative_average_per_date = SAA_1.groupby('Date')['TestsReported'].sum().rename('SAA_cumul_scalar')/n
cumulative_average_per_date.reset_index()
SAA_1 = SAA_1.merge(cumulative_average_per_date, left_on ='Day_before', right_on='Date', how='left')
SAA_1

,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar,SAA_t_1_vector,Day_before,SAA_cumul_scalar
0,Alabama,2020-04-12,2165,2020-04-13,NaN,NaN,2020-04-11,NaN
1,Louisiana,2020-04-12,1914,2020-04-13,NaN,NaN,2020-04-11,NaN
2,Arizona,2020-04-12,1119,2020-04-13,NaN,NaN,2020-04-11,NaN
3,Rhode Island,2020-04-12,1059,2020-04-13,NaN,NaN,2020-04-11,NaN
4,Maryland,2020-04-12,3015,2020-04-13,NaN,NaN,2020-04-11,NaN
...,...,...,...,...,...,...,...,...
7171,New Hampshire,2020-08-27,3952,2020-08-28,19515.0,4619.0,2020-08-26,19515.269231
7172,Nevada,2020-08-27,7673,2020-08-28,19515.0,5516.0,2020-08-26,19515.269231
7173,Nebraska,2020-08-27,5925,2020-08-28,19515.0,5288.0,2020-08-26,19515.269231
7174,Oregon,2020-08-27,6098,2020-08-28,19515.0,7248.0,2020-08-26,19515.269231


In [226]:
### df for SAA cumulative prescription

# vectoral SAA cumul

cumulative_sum_per_state = SAA_1.groupby('State')['TestsReported'].cuan() - SAA_1['TestsReported']
cumulative_sum_per_state
SAA_1['SAA_cumul_vector'] = cumulative_sum_per_state

AttributeError: 'SeriesGroupBy' object has no attribute 'cummean'

In [219]:
SAA_1

,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar,SAA_t_1_vector,Day_before,SAA_cumul_scalar,SAA_cumul_vector
0,Alabama,2020-04-12,2165,2020-04-13,NaN,NaN,2020-04-11,NaN,0
1,Louisiana,2020-04-12,1914,2020-04-13,NaN,NaN,2020-04-11,NaN,0
2,Arizona,2020-04-12,1119,2020-04-13,NaN,NaN,2020-04-11,NaN,0
3,Rhode Island,2020-04-12,1059,2020-04-13,NaN,NaN,2020-04-11,NaN,0
4,Maryland,2020-04-12,3015,2020-04-13,NaN,NaN,2020-04-11,NaN,0
...,...,...,...,...,...,...,...,...,...
7171,New Hampshire,2020-08-27,3952,2020-08-28,19515.0,4619.0,2020-08-26,19515.269231,306940
7172,Nevada,2020-08-27,7673,2020-08-28,19515.0,5516.0,2020-08-26,19515.269231,766374
7173,Nebraska,2020-08-27,5925,2020-08-28,19515.0,5288.0,2020-08-26,19515.269231,534360
7174,Oregon,2020-08-27,6098,2020-08-28,19515.0,7248.0,2020-08-26,19515.269231,750317


In [225]:
SAA_1[100:150]

,State,Date,TestsReported,Shift_Date,SAA_t_1_scalar,SAA_t_1_vector,Day_before,SAA_cumul_scalar,SAA_cumul_vector
100,Vermont,2020-04-13,398,2020-04-14,2151.0,471.0,2020-04-12,2151.384615,471
101,Arkansas,2020-04-13,293,2020-04-14,2151.0,625.0,2020-04-12,2151.384615,625
102,Illinois,2020-04-13,5728,2020-04-14,2151.0,3198.0,2020-04-12,2151.384615,3198
103,Delaware,2020-04-13,767,2020-04-14,2151.0,496.0,2020-04-12,2151.384615,496
104,Utah,2020-04-14,6461,2020-04-15,2193.0,3521.0,2020-04-13,2193.000000,5235
105,South Dakota,2020-04-14,0,2020-04-15,2193.0,0.0,2020-04-13,2193.000000,0
106,Florida,2020-04-14,14621,2020-04-15,2193.0,14472.0,2020-04-13,2193.000000,18990
107,Montana,2020-04-14,730,2020-04-15,2193.0,314.0,2020-04-13,2193.000000,746
108,Minnesota,2020-04-14,56,2020-04-15,2193.0,85.0,2020-04-13,2193.000000,113
109,Kansas,2020-04-14,1091,2020-04-15,2193.0,516.0,2020-04-13,2193.000000,1131
